<h1> UNILAB 2018 - Discentes, docentes, disciplinas e cursos 


Membros:
*   Lucas Brant - 
*   Luiz Felipe Nery - 2018054770 
*   Pedro Wildemberg - 
*   Tales Panoutsos - 

<br>

---



<h1>  Descrição dos dados</h1>
<p>Bla bla</p>
<br>



<h1>Diagrama ER</h1>

<h1>Diagrama Relacional</h1>
<h2> Forma Normalizada

* **cursos**(id_curso, nome_curso, nivel, modalidade, area_conhecimento)

* **docentes**(id_docente, nome_docente, sigla_unidade, situacao)

  docente.sigla_unidade -> unidade.sigla

* **unidades**(sigla, nome_unidade)

* **discentes**(id_discente, num_matricula, ano_ingresso, periodo_atual, sexo, idade, raca, pais, forma_ingresso, discente_situacao, beneficiado_auxilio, id_curso)

* **matriculas**(id_matricula_disciplina, resultado_disciplina, ano, periodo, id_disciplina, id_discente, media_final, numero_faltas)

  matricula.id_discente -> discente.id_discente

  matricula.id_docente -> docente.id_docente

  matricula.id_disciplina -> disciplina.id_disciplina

* **disciplinas**(id_disciplina, nome_disciplina, carga_horaria, id_docente)

  disciplinas.id_docente -> docentes.id_docentes


<br>

---


<h1>Baixando os dados

<br>
<p>Todos os dados são referentes ao ano de 2018

In [216]:
#matriculas
#Modelo:     !wget  -O .csv
!wget http://dadosabertos.unilab.edu.br:8001/csv/matriculas_2018.csv -O matriculas_2018.csv
!wget http://dadosabertos.unilab.edu.br:8001/csv/lista_discentes.csv -O alunos.csv
!wget http://dadosabertos.unilab.edu.br:8001/csv/cursos_graduacao.csv -O cursos.csv
!wget http://dadosabertos.unilab.edu.br:8001/csv/docentes.csv -O docentes.csv


--2019-11-09 00:50:17--  http://dadosabertos.unilab.edu.br:8001/csv/matriculas_2018.csv
Resolving dadosabertos.unilab.edu.br (dadosabertos.unilab.edu.br)... 200.129.19.44
Connecting to dadosabertos.unilab.edu.br (dadosabertos.unilab.edu.br)|200.129.19.44|:8001... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24083994 (23M) [application/octet-stream]
Saving to: ‘matriculas_2018.csv’

matriculas_2018.csv 100%[===================>]  22.97M  6.65MB/s    in 3.8s    

2019-11-09 00:50:21 (6.10 MB/s) - ‘matriculas_2018.csv’ saved [24083994/24083994]

--2019-11-09 00:50:22--  http://dadosabertos.unilab.edu.br:8001/csv/lista_discentes.csv
Resolving dadosabertos.unilab.edu.br (dadosabertos.unilab.edu.br)... 200.129.19.44
Connecting to dadosabertos.unilab.edu.br (dadosabertos.unilab.edu.br)|200.129.19.44|:8001... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1181703 (1.1M) [application/octet-stream]
Saving to: ‘alunos.csv’

alunos.csv          100%[========

In [217]:
!wget -c http://dadosabertos.unilab.edu.br/api/action/datastore_search?resource_id=89132ea3-318d-44e1-a269-2795ed168ae8&limit=1600 -O disciplinas.json

/bin/bash: -O: command not found
--2019-11-09 00:50:28--  http://dadosabertos.unilab.edu.br/api/action/datastore_search?resource_id=89132ea3-318d-44e1-a269-2795ed168ae8
Resolving dadosabertos.unilab.edu.br (dadosabertos.unilab.edu.br)... 200.129.19.44
Connecting to dadosabertos.unilab.edu.br (dadosabertos.unilab.edu.br)|200.129.19.44|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31885 (31K) [application/json]
Saving to: ‘datastore_search?resource_id=89132ea3-318d-44e1-a269-2795ed168ae8’

datastore_search?re 100%[===================>]  31.14K   193KB/s    in 0.2s    

2019-11-09 00:50:29 (193 KB/s) - ‘datastore_search?resource_id=89132ea3-318d-44e1-a269-2795ed168ae8’ saved [31885/31885]



In [0]:
!mv datastore_search?resource_id=89132ea3-318d-44e1-a269-2795ed168ae8 disciplinas.json

In [219]:
#Aquivos baixados (e outros)
!ls

alunos.csv  disciplinas.json  matriculas_2018.csv  unilab.sqlite
cursos.csv  docentes.csv      sample_data


--- 

<br>
<h1>Adicionando bibliotecas


In [0]:
import io
import sqlite3
import pandas as pd
import numpy as np

<br>

---

<br>

<h1>Editando os dados
<br>

<br>

<h2>Disciplinas

In [221]:
#json cru, obtido da requisicao
disciplinas = pd.read_json("disciplinas.json")

#Selecionamos somente os resultados na coluna "result" linha "records"
disciplinas = pd.DataFrame.from_dict(disciplinas["result"]["records"], orient='columns')
disciplinas = disciplinas.drop("",1)

disciplinas = disciplinas.drop("unidade",1)
disciplinas = disciplinas.drop("curso",1)
disciplinas = disciplinas.drop("horario",1)
disciplinas = disciplinas.drop("vagas",1)
disciplinas = disciplinas.drop("_id",1)
disciplinas = disciplinas.drop("local",1)
disciplinas = disciplinas.drop("id_unidade",1)

#cada disciplina so pode ser dada por um professor
disciplinas.drop_duplicates("codigo_disciplina", keep = 'first', inplace = True)

print(len(disciplinas))
disciplinas.head()

KeyError: ignored

In [0]:
disciplinas[disciplinas["codigo_disciplina"] == "CCAP0103"]

<br>

---

<br>

<h2>Matriculas</h2>





* **matriculas**(id_matricula_disciplina, resultado_disciplina, ano, periodo, id_disciplina, id_discente, media_final, numero_faltas, docente_nome)

  matricula.id_discente -> discente.id_discente

  matricula.id_docente -> docente.id_docente

  matricula.id_disciplina -> disciplina.id_disciplina

In [0]:
matriculas_denorm = pd.read_csv("matriculas_2018.csv")

matriculas = matriculas_denorm[['id_matricula_disciplina', 'resultado_disciplina', 'ano', 'periodo', 'codigo_disciplina', 'id_discente', 'media_final', 'numero_faltas']].copy()
print('Número de linhas: ',len(matriculas))
matriculas.drop_duplicates(inplace = True)
matriculas.head(6)

Número de linhas:  68014


,id_matricula_disciplina,resultado_disciplina,ano,periodo,codigo_disciplina,id_discente,media_final,numero_faltas
0,355799,APROVADO,2018,1,CCLH0025,b60a59793f8a6f6e82ebf9118834a73f,5.5,0.0
1,367655,APROVADO,2018,1,CCLH0025,878f6f53af90c86a083b81981efc8e54,7.1,0.0
2,385123,APROVADO,2018,2,CCLH0025,f5ea8a84a6d60c74972d3a26d6794131,9.3,8.0
3,367598,REPROVADO,2018,1,CCLH0025,4e3ab15c288d2b37b94c35b355f36672,0.0,0.0
4,382992,APROVADO,2018,2,CCLH0025,0bfd02874c1edd452e2b9fcec14c859e,8.2,12.0
6,367607,REPROVADO,2018,1,CCLH0025,74bfacfa56b05ae1442f80facab3161c,0.9,0.0


<br><h3>Consistente?</h3>

<p>Possui dulicatas?<p>

In [0]:
matriculas[['codigo_disciplina','id_discente','periodo', 'ano','resultado_disciplina','media_final','numero_faltas', 'id_matricula_disciplina']].duplicated().any()

False

<br><p>Possui chave?</p>
<p>Sim, a coluna id_matricula_disciplina</p><br>


<br>

---

<br>

<h2> Cursos

In [0]:
cursos = pd.read_csv("cursos.csv")
cursos = cursos.drop("Tipo de Oferta da Disciplina",1)
cursos = cursos.drop("Portaria Reconhecimento",1)
cursos = cursos.drop("DOU",1)
cursos = cursos.drop("Data Cadastro",1)
cursos = cursos.drop("Grau Acadêmico",1)
cursos = cursos.drop("Código INEP",1)
cursos = cursos.drop(20) #Retirando coluna que repetida
cursos = cursos.drop("Município",1)
print("Numero de de cursos: " + str(len(cursos)))
print("Tem duplicata usando Cursos e Modalidade Educação como chave? " + str((cursos[["Curso","Modalidade Educação"]]).duplicated().any()))
cursos.head()

Numero de de cursos: 41
Tem duplicata usando Cursos e Modalidade Educação como chave? True


,ID,Curso,Modalidade Educação,Área Conhecimento CNPQ
0,1,ADMINISTRAÇÃO PÚBLICA,A Distância,Ciências Sociais Aplicadas
1,2,ADMINISTRAÇÃO PÚBLICA,Presencial - Trimestral,Ciências Sociais Aplicadas
2,3,ADMINISTRAÇÃO PÚBLICA - SEMESTRAL,Presencial,Ciências Sociais Aplicadas
3,4,AGRONOMIA,Presencial,Ciências Agrárias
4,5,AGRONOMIA - TRIMESTRAL,Presencial - Trimestral,Ciências Agrárias


In [0]:
#Removendo as duplicatas
cursos.drop_duplicates(subset =["Curso","Modalidade Educação"],keep = 'first', inplace = True)
print("Tem duplicata usando Cursos e Modalidade Educação como chave? " + str((cursos[["Curso","Modalidade Educação"]]).duplicated().any()))
print("Numerode de cursos: " + str(len(cursos)))

Tem duplicata usando Cursos e Modalidade Educação como chave? False
Numerode de cursos: 37


O objetivo de nao haver duplicatas em cursos em relacao ao nome do curso e a modalidade de educação é porque dessa forma podemos fazer um mapeamento na tabela alunos para o ID de cada curso. Dessa forma, fazermos isso para deixar as tabelas na forma normal e evitar redundancias. 

<br>

---

<br>

<h2> Docentes

In [0]:
docentes = pd.read_csv("docentes.csv")
docentes = docentes.drop('categoria',1)
docentes = docentes.drop('denominacao',1)
docentes = docentes.drop('unidade',1)
print(len(docentes))
docentes.head() 

350


,nome,sigla,situação
0,ANDREA GOMES LINARD,ICS,Ativo Permanente
1,LEILANE BARBOSA DE SOUSA,ICS,Ativo Permanente
2,MARIA AUXILIADORA BEZERRA FECHINE,ICS,Ativo Permanente
3,ANA CAROLINE ROCHA DE MELO LEITE,ICS,Ativo Permanente
4,FLAVIA PAULA MAGALHÃES MONTEIRO,ICS,Ativo Permanente


In [0]:
print("Tamanho de docentes: " + str(len(docentes)))
print("Tamanho de docentes sem repetir nomes: " + str(len(docentes.groupby(["nome"]).count())))

# Sabemos que nome não é uma chave primaria muito boa, contudo, como nao temos as informacoes necessarias nas outras tabelas em que um docente aparece, nao conseguimos trocar a chave primaria para uma chave unica, por exemplo, uma numerica.

Tamanho de docentes: 350
Tamanho de docentes sem repetir nomes: 346


<br>

---

<br>

<h2> Alunos

In [0]:
alunos = pd.read_csv("alunos.csv")
alunos.groupby('estado').count()

,id_discente,matricula,ano_ingresso,periodo_ingresso,periodo_atual,sexo,idade,raça,pais,curso,nivel,modalidade,polo,forma_ingresso,discente_situacao,beneficiado_auxilio,prazo_conclusao,data_colacao_grau
estado,,,,,,,,,,,,,,,,,,
Bahia,1161,1161,1161,1161,1059,1161,1161,1161,1161,1161,1161,1161,102,1161,1161,1161,1059,0
Ceará,5088,5088,5088,5088,4225,5088,5088,5080,5088,5088,5088,5088,1094,5088,5088,5088,4225,0
Pará,41,41,41,41,0,41,41,41,41,41,41,41,41,41,41,41,0,0


In [0]:
#tem q tirar os alunos do para
#todos eles tem o curso bugado: SAÚDE DA FAMÍLIA - 2017.2
((alunos[alunos['estado'] == 'Pará']).groupby("curso").count()).head()

#Observe que todos os alunos do Pará fazem o emso curso. Este curso não esta na tabela de cursos.

,id_discente,matricula,ano_ingresso,periodo_ingresso,periodo_atual,sexo,idade,raça,pais,nivel,modalidade,estado,polo,forma_ingresso,discente_situacao,beneficiado_auxilio,prazo_conclusao,data_colacao_grau
curso,,,,,,,,,,,,,,,,,,
SAÚDE DA FAMÍLIA - 2017.2,41,41,41,41,0,41,41,41,41,41,41,41,41,41,41,41,0,0


In [0]:
#Todos os cursos que sao feitos por esses alunos -> Muitos deles são cursos antigos (de antes de 2018) que não estao 
#em nossa tabela de cursos, uma vez que temos somente informações sobre os cursos de 2018
print("numero de cursos: " + str(len(alunos.groupby('curso').count().index)))
alunos.groupby('curso').count().index

numero de cursos: 41


Index(['ADMINISTRAÇÃO PÚBLICA', 'ADMINISTRAÇÃO PÚBLICA - SEMESTRAL',
       'AGRONOMIA', 'ANTROPOLOGIA', 'CIÊNCIAS BIOLÓGICAS',
       'CIÊNCIAS DA NATUREZA E MATEMÁTICA', 'CIÊNCIAS SOCIAIS', 'ENFERMAGEM',
       'ENGENHARIA DA COMPUTAÇÃO', 'ENGENHARIA DE ENERGIAS',
       'ENSINO E FORMAÇÃO DOCENTE', 'ESTUDOS DA LINGUAGEM', 'FARMÁCIA',
       'FÍSICA', 'GESTÃO EM SAÚDE - OFERTA 2013.1', 'GESTÃO EM SAÚDE 2017.1',
       'GESTÃO EM SAÚDE 2017.2', 'GESTÃO EM SAÚDE 2018.2',
       'GESTÃO PÚBLICA - 2017.1', 'GESTÃO PÚBLICA - 2017.2',
       'GESTÃO PÚBLICA MUNICIPAL - 2017.2', 'HISTÓRIA', 'HUMANIDADES',
       'LETRAS -  LÍNGUA PORTUGUESA', 'LETRAS - LÍNGUA INGLESA',
       'LETRAS - LÍNGUA PORTUGUESA', 'MATEMÁTICA',
       'MESTRADO ACADÊMICO EM ANTROPOLOGIA UFC/UNILAB',
       'MESTRADO ACADÊMICO EM ENERGIA E AMBIENTE',
       'MESTRADO ACADÊMICO EM ENFERMAGEM',
       'MESTRADO PROFISSIONAL EM MATEMÁTICA EM REDE NACIONAL', 'PEDAGOGIA',
       'PROGRAMA DE PÓS-GRADUAÇÃO INTERDISCIPLINAR

In [0]:
alunos = pd.read_csv("alunos.csv")
discentes = pd.read_csv("alunos.csv")

discentes = discentes.drop('polo',1)
discentes = discentes.drop('prazo_conclusao',1)
discentes = discentes.drop('periodo_ingresso',1)
discentes = discentes.drop('data_colacao_grau',1)
discentes = discentes.drop('nivel',1)
discentes = discentes.drop('estado',1)

In [0]:
curso_id = []
for i in range (len(alunos)):
  match = False
  for j in range (len(cursos)):
    if alunos["curso"].iloc[i] == cursos["Curso"].iloc[j] and alunos["modalidade"].iloc[i] == cursos["Modalidade Educação"].iloc[j]:
      id = cursos["ID"].iloc[j]
      curso_id.append(id)
      match = True
  if not(match):
    discentes = discentes.drop(alunos.index[i])

In [0]:
discentes["id_curso"] = curso_id

In [0]:
discentes = discentes.drop('curso',1)
discentes = discentes.drop('modalidade',1)

In [0]:
discentes.head()

,id_discente,matricula,ano_ingresso,periodo_atual,sexo,idade,raça,pais,forma_ingresso,discente_situacao,beneficiado_auxilio,id_curso
0,0003ba85ff2e992a07361dbe7e28d0bf,2018102***,2018,4.0,M,24,Negra,Angola,SELEÇÃO DE ESTUDANTES ESTRANGEIROS,ATIVO,f,33
1,00082dd79190f2e00b15da27afcae03b,2019101***,2019,2.0,F,21,Parda,Brasil,REINGRESSO SEGUNDO CICLO,ATIVO,f,35
2,0015735285c7b9cc5d97826f4f7a2564,2015106***,2015,10.0,M,29,Negra,Guiné Bissau,REOPCAO,FORMANDO,t,16
3,0016b115d8bcfce421766b146f40a388,2013102***,2013,14.0,M,39,Parda,Brasil,VESTIBULAR,ATIVO,f,1
4,0028bbd61a14e1a811dfc2617d5e2544,2017107***,2017,6.0,F,20,Negra,Brasil,SiSU,ATIVO,f,8


<br>
<br>

---

<br>

<h1> Criando o banco de dados

**'df' deve ser usada como variavel temporaria**

In [0]:
!rm unilab.sqlite
!ls

alunos.csv  disciplinas.json  matriculas_2018.csv
cursos.csv  docentes.csv      sample_data


In [0]:
#criando e conectando banco
conn = sqlite3.connect("unilab.sqlite") #se o arquivo nao existir, um novo e gerado
cursor = conn.cursor()
conn.execute("PRAGMA foreign_keys = 1") #garante que cada foreign key existe na tabela referenciada, se nao existir a insercao da erro
!ls

alunos.csv  disciplinas.json  matriculas_2018.csv  unilab.sqlite
cursos.csv  docentes.csv      sample_data


---

<br>

<h2>Criando tabela Unidades no bando de dados 

In [0]:
#criando tabela UNIDADE
cursor.execute('''CREATE TABLE UNIDADE
         (sigla TEXT PRIMARY KEY NOT NULL,
         unidade TEXT NOT NULL);''')

In [0]:
#inserindo dados na tabela unidade
df = pd.read_csv('docentes.csv')
df = df[['sigla', 'unidade']].drop_duplicates() #tuplas tem de ser unicas
df.to_sql('UNIDADE', conn, if_exists='append', index=False)
unidades_sql = df.copy()

In [0]:
unidades_sql

,sigla,unidade
0,ICS,INSTITUTO DE CIÊNCIAS DA SAÚDE
36,ICEN,INSTITUTO DE CIÊNCIAS EXATAS E DA NATUREZA
74,ICSA,INSTITUTO DE CIÊNCIAS SOCIAIS APLICADAS
93,IDR,INSTITUTO DE DESENVOLVIMENTO RURAL
122,IEDS,INSTITUTO DE ENGENHARIAS E DESENVOLVIMENTO SUS...
146,IH,INSTITUTO DE HUMANIDADES
220,IHLM,INSTITUTO DE HUMANIDADES E LETRAS/MALÊS
312,ILL,INSTITUTO DE LINGUAGENS E LITERATURAS
345,UNILAB,UNIVERSIDADE DA INTEGRAÇÃO INTERNACIONAL DA LU...


---

<br>

<h2> Criando tabela Docente no banco de dados

In [0]:
#criando tabela DOCENTE
cursor.execute('''CREATE TABLE DOCENTE
         (nome TEXT NOT NULL,
         situacao TEXT NOT NULL,
         sigla TEXT NOT NULL,
         PRIMARY KEY (nome, sigla),
         FOREIGN KEY (sigla)
          REFERENCES UNIDADE(sigla));''')

#coloquei sigla e nome como chave primaria, aparentemente tem professores com nomes iguais em institutos diferentes

In [0]:
#Inserindo dados na tabela DOCENTE
df = pd.read_csv('docentes.csv')
df = df[['nome', 'sigla', 'situação']].drop_duplicates()
df.columns = ['nome', 'sigla', 'situacao'] #troquei os nomes do dataframe pra ficar igual da tabela
df.to_sql('DOCENTE', conn, if_exists='append', index=False) #if_exists=append garante que vai inserir na tabela existente
                                                             #index=False evita que uma coluna adicional seja criada 

In [0]:
docentes_sql = df.copy()
docentes_sql

,nome,sigla,situacao
0,ANDREA GOMES LINARD,ICS,Ativo Permanente
1,LEILANE BARBOSA DE SOUSA,ICS,Ativo Permanente
2,MARIA AUXILIADORA BEZERRA FECHINE,ICS,Ativo Permanente
3,ANA CAROLINE ROCHA DE MELO LEITE,ICS,Ativo Permanente
4,FLAVIA PAULA MAGALHÃES MONTEIRO,ICS,Ativo Permanente
...,...,...,...
344,SUELI DA SILVA SARAIVA,ILL,Ativo Permanente
345,JOSE BERTO NETO,UNILAB,Ativo Permanente
346,MARIA ELIAS SOARES,UNILAB,Nomeado Cargo Comissão
347,MARIA DE FATIMA BARBOSA COELHO,UNILAB,Aposentado


<h3>Consulta de teste

In [0]:
#query de exemplo com os dados acima
df = pd.read_sql("SELECT * FROM DOCENTE WHERE sigla = 'ICS'", conn)
df

,nome,situacao,sigla
0,ANDREA GOMES LINARD,Ativo Permanente,ICS
1,LEILANE BARBOSA DE SOUSA,Ativo Permanente,ICS
2,MARIA AUXILIADORA BEZERRA FECHINE,Ativo Permanente,ICS
3,ANA CAROLINE ROCHA DE MELO LEITE,Ativo Permanente,ICS
4,FLAVIA PAULA MAGALHÃES MONTEIRO,Ativo Permanente,ICS
5,RAFAELLA PESSOA MOREIRA,Ativo Permanente,ICS
6,GILVAN FERREIRA FELIPE,Ativo Permanente,ICS
7,MONALIZA RIBEIRO MARIANO,Ativo Permanente,ICS
8,PAULA MARCIANA PINHEIRO DE OLIVEIRA,Ativo Permanente,ICS
9,EMILIA SOARES CHAVES ROUBERTE,Ativo Permanente,ICS


---

<br>

<h2> Criando tabela Cursos no banco de dados

In [0]:
#criando tabela CURSOS

cursor.execute('''CREATE TABLE CURSOS 
         (id_curso INTEGER PRIMARY KEY NOT NULL, 
         nome_curso TEXT NOT NULL, 
         modalidade TEXT NOT NULL, 
         area_conhecimento TEXT NOT NULL); ''')

#coloquei sigla e nome como chave primaria, aparentemente tem professores com nomes iguais em institutos diferentes

In [0]:
df = cursos.copy()
if('Município' in df.columns):
  df = df.drop('Município',1)
print(df.columns)
df.columns = ['id_curso', 'nome_curso', 'modalidade', 'area_conhecimento'] #troquei os nomes do dataframe pra ficar igual da tabela
print(df.columns)

Index(['ID', 'Curso', 'Modalidade Educação', 'Área Conhecimento CNPQ'], dtype='object')
Index(['id_curso', 'nome_curso', 'modalidade', 'area_conhecimento'], dtype='object')


In [0]:
#Inserindo dados na tabela CURSOS
df.to_sql('CURSO', conn, if_exists='append', index=False) #if_exists=append garante que vai inserir na tabela existente
                                                             #index=False evita que uma coluna adicional seja criada 

<h3> Consulta Teste

In [0]:
df = pd.read_sql("SELECT * FROM CURSO WHERE area_conhecimento = 'Ciências Sociais Aplicadas'", conn)
df

,id_curso,nome_curso,modalidade,area_conhecimento
0,1,ADMINISTRAÇÃO PÚBLICA,A Distância,Ciências Sociais Aplicadas
1,2,ADMINISTRAÇÃO PÚBLICA,Presencial - Trimestral,Ciências Sociais Aplicadas
2,3,ADMINISTRAÇÃO PÚBLICA - SEMESTRAL,Presencial,Ciências Sociais Aplicadas


---

<br>

<h1>Consultas</h1>
<h2>Consultas usando Projeção e seleção </h2>
<br>
<h3>Consulta 1

<h3>Consulta 2

---

<br>

<h2>Consultas envolvendo junção de duas relações 
<br>
<h3>Consulta 1 

<h3>Consulta 2

<h3>Consulta 3

---

<br>

<h2>Consultas envolvendo junção de três ou mais relações

<br>

<h3>Consulta 1

<h3>Consulta 2

<h3>Consulta 3

---

<br>

<h2>Consultas envolvendo agregação sobre junção de duas ou mais relações

<br>

<h3>Consulta 1


<h3>Consulta 2

---

<br>

<h1>Autoavaliação dos membros</h1>








# Divisão quanto ao tratamento de dados:
Luiz: Docentes e disciplinas

Lucas: alunos

Tales: matriculas

Pedro: Cursos

<br>
<br>

Obs1.: Os nomes acima seguem o padrao do dataframe que pode ser vista abaixo

Obs2.: Mantenham o padrão dos nomes

Obs3.: Esse notebook será o entregavel final, então a medida que foram fazendo as coisas vao documentando para ficar mais facil

Obs4.: Se nao for possivel alterar esse notebook no original, salvem ele e trabalhem separadamente e depois a gente junta tudo